In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from deepquantum.nn.modules.quanv import QuanConv2D

In [13]:
BATCH_SIZE = 5
EPOCHS = 30     # Number of optimization epochs
n_train = 500    # Size of the train dataset
n_test = 5   # Size of the test dataset

SAVE_PATH = "../tests/"  # Data saving folder
PREPROCESS = True           # If False, skip quantum processing and load data from SAVE_PATH
seed = 23
np.random.seed(seed)        # Seed for NumPy random number generator
torch.manual_seed(seed)     # Seed for TensorFlow random number generator

In [14]:
DEVICE = torch.device("cpu")

train_dataset = datasets.MNIST(root="./data",
                               train=True,
                               download=True,
                               transform=transforms.ToTensor())

train_dataset.data = train_dataset.data[:n_train]
train_dataset.targets = train_dataset.targets[:n_train]

test_dataset = datasets.MNIST(root="./data",
                              train=False,
                              transform=transforms.ToTensor())

test_dataset.data = test_dataset.data[:n_test]
test_dataset.targets = test_dataset.targets[:n_test]

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.quan1 = QuanConv2D(n_qubits=4, stride=2, kernel_size=2)
        self.fc1 = nn.Linear(14*14*4, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 10)

    def forward(self, x):
        x = self.quan1(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        return x

In [16]:
model = Net().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_func = nn.CrossEntropyLoss()


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=False)

In [17]:
loss_list = []

model.train().to(DEVICE)
for epoch in range(EPOCHS):
    total_loss = []
    for batch_idx, (data, target) in enumerate(train_loader):
        target = target.to(DEVICE)
        optimizer.zero_grad()
        data = data.to(torch.float32).to(DEVICE)

        # Forward pass
        output = model(data).to(DEVICE)

        # Calculating loss
        loss = loss_func(output, target).to(DEVICE)

        # Backward pass
        loss.backward()

        # Optimize the weights
        optimizer.step()

        total_loss.append(loss.item())
    loss_list.append(sum(total_loss) / len(total_loss))
    print('Training [{:.0f}%]\tLoss: {:.4f}'.format(100. * (epoch + 1) / EPOCHS, loss_list[-1]))

torch.save(model.state_dict(), "quan20211216.pt")

Training [3%]	Loss: 2.2116
Training [7%]	Loss: 1.9365
Training [10%]	Loss: 1.8016
Training [13%]	Loss: 1.8617
Training [17%]	Loss: 1.8549
Training [20%]	Loss: 1.8149
Training [23%]	Loss: 1.7804
Training [27%]	Loss: 1.6977
Training [30%]	Loss: 1.6205
Training [33%]	Loss: 1.5213
Training [37%]	Loss: 1.4945
Training [40%]	Loss: 1.4658
Training [43%]	Loss: 1.3726
Training [47%]	Loss: 1.2667
Training [50%]	Loss: 1.2686
Training [53%]	Loss: 1.2119
Training [57%]	Loss: 1.1255
Training [60%]	Loss: 1.1200
Training [63%]	Loss: 1.2115
Training [67%]	Loss: 1.2919
Training [70%]	Loss: 1.1138
Training [73%]	Loss: 1.0879
Training [77%]	Loss: 1.0773
Training [80%]	Loss: 1.1411
Training [83%]	Loss: 1.4680
Training [87%]	Loss: 1.2237
Training [90%]	Loss: 1.0615
Training [93%]	Loss: 1.0408
Training [97%]	Loss: 1.0114
Training [100%]	Loss: 0.9939


In [19]:
model.load_state_dict(torch.load("quan20211216.pt"))

model.eval()
with torch.no_grad():
    correct = 0
    total_loss = []
    for batch_idx, (data, target) in enumerate(test_loader):
        data = data.to(torch.float32).to(DEVICE)
        target = target.to(DEVICE)
        output = model(data).to(DEVICE)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        loss = loss_func(output, target)
        total_loss.append(loss.item())
    print('Performance on test data:\n\tLoss: {:.4f}\n\tAccuracy: {:.1f}%'.format(
        sum(total_loss) / len(total_loss),
        correct / len(test_loader) * 100 / BATCH_SIZE)
        )

Performance on test data:
	Loss: 0.6726
	Accuracy: 60.0%
